### 从log文件输出信息  
`variable`  
`print`  
`variable`用于变量定义
`variable E0 equal etotal`   
`variable E_prefect equal ${E0}`  
由于`etotal`是系统的总能量值（特殊的词，LAMMPS中已定义为系统总能量，其它热力学信息特定词可详查`thermo_style`命令），会随着命令的执行而不断变化，所以这里连用两个`variable`命令来记录第一次优化后系统的能量值。  
`variable  N equal count(all)`  
`variable  N0 equal ${N}`  
同理，连用两个`variable`命令来记录第一次优化后的总原子数，`count（all）`表示计算总原子数目。  
而第二次（最后一次）优化后，由于`etotal`不再变化，则只需用一个`variable`命令记录。  

In [5]:
%%writefile ex1.in
units metal
atom_style atomic 
boundary p p p 
neighbor 0.3 bin

lattice bcc 3.168
region box block  0 10 0 10 0 10
create_box 1 box
create_atoms 1 box

pair_style eam/alloy
pair_coeff * * W_zhou.eam.alloy W

thermo 10
dump 1 all custom 1 vacancy.xvz id type x y z

minimize 1.0e-12 1.0e-13 10000 10000

# 记录第一次优化后的能量值与总原子数目
variable E0 equal etotal
variable E_prefect equal ${E0}
variable N equal count(all)
variable N0 equal ${N}

region 2 sphere 5.0 5.0 5.0 0.1 side in
delete_atoms region 2

minimize 1.0e-12 1.0e-13 10000 10000
variable E_defect equal etotal
variable Ev equal (${E_defect}-((${N0}-1)/${N0})*${E_prefect})

print "ALL done"
print "Vacancy formation energy = ${Ev} ev"

Overwriting ex1.in


In [6]:
!"E:/LAMMPS/bin/lmp" < ex1.in

LAMMPS (29 Sep 2021 - Update 2)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 3.1680000 3.1680000 3.1680000
Created orthogonal box = (0.0000000 0.0000000 0.0000000) to (31.680000 31.680000 31.680000)
  1 by 1 by 1 MPI processor grid
Created 2000 atoms
  using lattice units in orthogonal box = (0.0000000 0.0000000 0.0000000) to (31.680000 31.680000 31.680000)
  create_atoms CPU = 0.001 seconds
Reading eam/alloy potential file W_zhou.eam.alloy with DATE: 2007-10-12
Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 8.1925
  ghost atom cutoff = 8.1925
  binsize = 4.09625, bins = 8 8 8
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair eam/alloy, perpetual
      attributes: half, newton on
      pair build: half/bin/atomonly/newton
      stencil: half/bin/3d
      bin: standar